In [1]:
import pandas as pd
from pathlib import Path
import numpy as np

/users/3/hassa940/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
root = Path("results_per_section")

In [3]:
dfs = [pd.read_csv(f"results_per_section/{fp.name}") for fp in root.glob("*.csv")]

In [4]:
raw_df = pd.concat(dfs, ignore_index=True)

In [5]:
chk = (
    raw_df.groupby(["freq_subset", "rcond", "measure"], as_index=False)
        .agg(
            n_rows      = ("overall_accuracy", "size"),
            sum_accuracy= ("overall_accuracy", "sum"),
            avg_accuracy= ("overall_accuracy", "mean")
        )
)

In [6]:
summary_df = pd.read_csv("merged_results/merged_results_measurements.csv")

In [7]:
tol = 1e-12                                      # numerical tolerance

In [8]:
# A) Every combination should appear exactly once

assert len(summary_df) == 100, "Expected 100 unique (freq,rcond,measure) rows."

In [9]:
# B) Join summaries and compare all numeric columns

merge = summary_df.merge(chk, on=["freq_subset", "rcond", "measure"], suffixes=("_saved", "_chk"), how="outer")
assert len(merge) == 100, "Mismatch in key sets between saved and recomputed summaries."

In [10]:
# C) Verify equality of counts, sums, means (within tolerance)

cols_to_check = ["n_rows", "sum_accuracy", "avg_accuracy"]
for col in cols_to_check:
    diff = np.abs(merge[f"{col}_saved"] - merge[f"{col}_chk"])
    assert (diff < tol).all(), f"Discrepancy found in {col}: max diff = {diff.max()}"

In [11]:
# D) Quick logical check: avg ≈ sum / n_rows in the saved file

ratio_diff = np.abs(summary_df["avg_accuracy"] - summary_df["sum_accuracy"] / summary_df["n_rows"])
assert (ratio_diff < tol).all(), f"Saved avg != sum/n: max diff = {ratio_diff.max()}"

print("All sanity checks passed: counts, sums, and averages are consistent.")

All sanity checks passed: counts, sums, and averages are consistent.
